### IMPORTING LIBRARIES

In [10]:
import warnings
warnings.filterwarnings('ignore')

In [11]:
#Exploratory Data Analysis Libraries
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

In [12]:
!pip install optuna
#Predictive Models
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest
from sklearn.pipeline import make_pipeline
import lightgbm as lgb
from sklearn.metrics import mean_squared_log_error
import xgboost as xgb
from xgboost import plot_importance
import optuna
from sklearn.model_selection import train_test_split

### PREPARING DATA FOR PROCESSING

In [13]:
def separate_train(df):

  #Separate train and test data
  df_dummies = pd.get_dummies(df, drop_first=True)

  train_start = '2015-01-01'
  train_end = '2017-08-15'

  train_data = df_dummies[(df_dummies['date'] >= train_start) & (df_dummies['date'] <= train_end)]

  feature_columns = [col for col in df_dummies.columns if col not in ['sales', 'date']]

  return train_data

In [14]:
def separate_test(df):

  #Separate train and test data
  df_dummies = pd.get_dummies(df, drop_first=True)

  test_start = '2017-08-16'
  test_end = '2017-08-31'

  test_data = df_dummies[(df_dummies['date'] >= test_start) & (df_dummies['date'] <= test_end)]

  return test_data

In [15]:
def prepare_data(df):

  #Import data
  train = separate_train(df)
  test = separate_test(df)

  #Prepare Features and Target
  # Target variable
  y = train['sales']

  # Features to use (add/remove as needed)
  # Drop columns not used as features
  drop_cols = ['id', 'date', 'sales', 'family']  # keep family_encoded

  # Select all other columns as features
  features = [col for col in train.columns if col not in drop_cols]

  # Prepare training and test data
  X = train[features]
  X_test = test[features]

  return X, y, X_test

### TRAINING MODEL

In [16]:
def train_model(df):

  #Import data
  X, y, X_test = prepare_data(df)

  #Train/Test Split
  X_train, X_val, y_train, y_val = train_test_split(

      X, y, test_size=0.2, shuffle=False

      )  # No shuffling due to time-series nature

    #Train XGBoost Model
  model = xgb.XGBRegressor(
    n_estimators=300,
    learning_rate=0.1,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
  )

  model.fit(X_train, y_train)

  return model